This sample code is mainly about pre-processing the Twitter data that is collected by calling Twitter API, and it's a class project that I did with guidance from my professor. The code hasn't been run in this notebook but should be error-free and executable. The purpose of this notebook is preparing the tweets for further steps, such as topic modeling and text mining.

Import the libraries and functions needed.

In [ ]:
import numpy as np
from tweepy.streaming           import StreamListener
from tweepy                     import OAuthHandler
from tweepy                     import Stream
from time                       import strftime
from json                       import loads, dumps
from pickle                     import dump
from time                       import strftime
from nltk.corpus                import TwitterCorpusReader
from nltk.tokenize              import TweetTokenizer
from unicodedata                import category
from sklearn.base               import BaseEstimator, TransformerMixin
from nltk                       import pos_tag, WordNetLemmatizer
from nltk.corpus                import stopwords
from nltk.corpus                import wordnet as wn
from sklearn.feature_extraction import DictVectorizer

Set up the working directory (below is just the directory I used).

In [ ]:
grp_dir='/Users/nicoleli/Desktop/textMining/'

Save the API into variables

In [ ]:
consumer_key        = u'2k7nmVtEUOhCjsxC3MqQxd2h4'
consumer_secret     = u'N3evtiwz5IJNjvxTPL3zPFufy8UE9JXdoVh7KGNUWYidOd5PBD'
access_token        = u'994799344674099201-cwx454YZWBC66WOJWkctoq0rbb3arco' 
access_token_secret = u'ug8CTfeF2AbbBZhtH3nA76P3nMvvmsRHpwxV53R7MbAXx' 

Authenticate Twitter account
The following code calls the api to connect to the designated Twitter account.

In [ ]:
from tweepy import OAuthHandler, API

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = API(auth)

Define the class that reads the streaming data from Twitter and saves the data in files.

In [ ]:
class StdOutListener(StreamListener):
  def __init__(self, path=grp_dir, file='tweet_data'):
    self.tweet_data = []
    self.tweet_count = 0
    self.path = path
    self.file = file
    self.full = self.path+self.file+"_"+strftime("%Y%m%d%H%M%S")
        
  def on_data(self, data): # counts the number of tweetsand index the tweets
    self.tweet_count += 1
    print("on_data", self.tweet_count)
    try:
      tweet = loads(data)
      print(tweet['text'])
    
      self.tweet_data.append(tweet)
    except Exception as Argument:
      print ("Exception:\n", Argument)
    return True
  
  def save_data(self): # saves the tweets to specified paths
    print("Saving to:",        self.full+".pkl")
    dump(self.tweet_data, open(self.full+".pkl", "wb"))

    print("Saving to:",self.full+".jsonl")
    with          open(self.full+".jsonl", 'w') as fd:
      for twt in self.tweet_data:
        fd.write(dumps(twt)+"\n")
  
  def on_error(self, status):
    print(status)

Create an StdOutListener object called twitter_listener so that the tweets can be saved into files called twitter_file_1.jsonl/.pkl.

In [ ]:
twitter_listener = StdOutListener(file='twitter_file_1')

Collect the tweets that are related to a specified keyword, for example "chairs". This step takes time to run, and requires to be stopped manually when enough tweets are collected.

In [ ]:
stream = Stream(auth, twitter_listener)
stream.filter(track=['chairs'])

Save the data just collected into files that are specified above.

In [ ]:
twitter_listener.save_data()

Define functions to filter/clean the tweets, and they are used in the AltTwitterCorpusReader class which is defined below.

In [ ]:
import re
def re_filter(tweet,re_list): # the function is only used to remove tokens which are links currently as defined below
    r = [re.compile(x) for x in re_list]
    for re_comp in r:
        tweet=filter(re_comp.match, tweet)
    return tweet
      
def is_punct(token): #True if every character is punctuation
    return all(category(char).startswith('P') for char in token)

def wnpos(tag): #Return the WordNet POS tag
    return {'n': wn.NOUN,
            'v': wn.VERB,
            'r': wn.ADV,
            'j': wn.ADJ
           }.get(tag[0].lower(), wn.ADJ_SAT)

Define AltTwitterCorpusReader class that can clean the tweets, including removing punctuations, urls, stop words and etc., tokenizing, lemmatizing and vectorizing the tweets. The methods implement the steps in vectorizing the corpus.

In [ ]:
class AltTwitterCorpusReader(TwitterCorpusReader):

    lemmatize = WordNetLemmatizer().lemmatize
     
    # assign default parameters    
    def __init__(self, tag_list      =['n','v','r','j'],
                       base_stopwords=set(stopwords.words('english')),
                       more_stopwords=set([]),
                       re_list       =['^(?!http)'],
                       min_len       =4,
                       **kwargs):
        TwitterCorpusReader.__init__(self,**kwargs)
        self.stopwords = base_stopwords.union(more_stopwords)
        self.tag_list  = tag_list
        self.re_list   = re_list
        self.min_len   = min_len
        
    def tokenize(self):
        self.tokens_   = TwitterCorpusReader.tokenized(self)
        return self
        
    def filter_tokens(self): # filters out the urls, punctuations, words that are shorter than minimum length and stopwords
        self.tokens_filtered_regexp_    = [list(re_filter(tweet,self.re_list)) 
                                           for tweet in self.tokens_]
        self.tokens_filtered_punct_     = [[token for token 
                                            in tweet 
                                            if not is_punct(token)
                                           ] for tweet in self.tokens_filtered_regexp_]
        self.tokens_filtered_min_len_   = [[token for token 
                                            in tweet 
                                            if len(token)>=self.min_len
                                           ] for tweet in self.tokens_filtered_punct_]
        self.tokens_filtered_stopwords_ = [[token for token 
                                            in tweet 
                                            if token not in self.stopwords
                                           ] for tweet in self.tokens_filtered_min_len_]
        self.tokens_filtered_           = self.tokens_filtered_stopwords_
        return self
      
    def add_pos(self):
        self.tokens_pos_                = [[(token, wnpos(tag)) for (token,tag) 
                                            in  pos_tag(tweet)
                                           ] for tweet in self.tokens_filtered_]
        return self
      
    def filter_pos(self): # only keeps the words of specified forms
        self.tokens_pos_filtered_       =  [[(token, tag) for (token,tag) 
                                             in tweet if  tag in self.tag_list
                                            ] for tweet in self.tokens_pos_]
        return self
        
    def normalize(self): # transforms the words back into their original form according to the context
        self.tokens_normalized_         = [[(self.lemmatize(token, tag), tag) 
                                            for (token, tag) in tweet
                                           ] for tweet in self.tokens_pos_filtered_]
        return self

    def vectorize(self): 
        self.dictionary_                = [{token:1 for (token,tag) in tweet
                                           } for tweet in self.tokens_normalized_]
        v = DictVectorizer(dtype =np.int16,
                           sparse=False)
        self.vectorized_ = v.fit_transform(self.dictionary_)
        self.vec_vocabulary_ = v.vocabulary_
        self.vec_feature_names_ =v.feature_names_
        return self

Create an AltTwitterCorpusReader object called twitter_reader with specified arguments, which would be the parameters for the functions called to clean the tweets.

In [ ]:
twitter_reader = AltTwitterCorpusReader(root          =grp_dir,
                                       fileids       ='twitter_file_1.jsonl',
                                       word_tokenizer=TweetTokenizer(preserve_case=False,strip_handles=True),
                                       re_list       =['^(?!http)'],
                                       more_stopwords=set([]),
                                       min_len       =4,
                                       tag_list      =['n','j','v','r']
                                      )

Call the methods in the AltTwitterCorpusReader class step by step, the result from the previous step is stored in the objectand is used in the next step.

In [ ]:
twitter_reader.tokenize()     \
              .filter_tokens()\
              .add_pos()      \
              .filter_pos()   \
              .normalize()    \
              .vectorize()

Display the first element/item from the result of each step in order to check if the results satisfy the expectation.

In [ ]:
n = 0
print('\ntokens',twitter_reader.tokens_[n])
print('\ntokens filtered',twitter_reader.tokens_filtered_[n])
print('\ntokens with pos',twitter_reader.tokens_pos_[n])
print('\npos filtered',twitter_reader.tokens_pos_filtered_[n])
print('\nnormalized',twitter_reader.tokens_normalized_[n])
print('\ndictionary',twitter_reader.dictionary_[n])
print('\nvectorized',twitter_reader.vectorized_[n])